In [1]:
from data import ModelNet40
dset = ModelNet40(grid=True)
[el.shape for el in dset[0]]

[torch.Size([3, 224, 224]), torch.Size([3, 16, 64])]

In [2]:
import torch
imgs = torch.stack([dset[i][0] for i in range(8)])
points = torch.stack([dset[i][1] for i in range(8)])

In [ ]:
imgs.shape, points.shape

In [ ]:
from model import full_conv_default
net = full_conv_default()
net

In [ ]:
%time [y.shape for y in net.eval()(imgs[0:1])]

In [3]:
from model import full_conv_default
from torch.optim import Adam
from model.util import grid_to_points
from model.loss import grid_chamfer_distance, depth_loss, chamfer_distance
epochs = 100
net = full_conv_default()
net = net.train()
opt = Adam(net.parameters())

for i in range(epochs):
    opt.zero_grad()
    depth, grid = net(imgs)
    dloss = depth_loss(depth, points)
    # gloss = chamfer_distance(grid_to_points(grid), grid_to_points(points), sqrt=True)
    gloss = grid_chamfer_distance(grid, points, (1, 4))
    (dloss + gloss).backward()
    opt.step()
    print(dloss, gloss)    
    #with torch.no_grad():
    #    print('Chamfer distance', chamfer_distance(grid_to_points(grid), grid_to_points(points), sqrt=True), '\n')
    

tensor(0.3797, grad_fn=<MeanBackward0>) tensor(0.5250, grad_fn=<AddBackward0>)
tensor(0.1187, grad_fn=<MeanBackward0>) tensor(0.3487, grad_fn=<AddBackward0>)
tensor(0.0872, grad_fn=<MeanBackward0>) tensor(0.2820, grad_fn=<AddBackward0>)
tensor(0.0826, grad_fn=<MeanBackward0>) tensor(0.2498, grad_fn=<AddBackward0>)
tensor(0.0747, grad_fn=<MeanBackward0>) tensor(0.2233, grad_fn=<AddBackward0>)
tensor(0.0712, grad_fn=<MeanBackward0>) tensor(0.2197, grad_fn=<AddBackward0>)
tensor(0.0684, grad_fn=<MeanBackward0>) tensor(0.2126, grad_fn=<AddBackward0>)
tensor(0.0667, grad_fn=<MeanBackward0>) tensor(0.2021, grad_fn=<AddBackward0>)
tensor(0.0639, grad_fn=<MeanBackward0>) tensor(0.1890, grad_fn=<AddBackward0>)
tensor(0.0634, grad_fn=<MeanBackward0>) tensor(0.1827, grad_fn=<AddBackward0>)
tensor(0.0614, grad_fn=<MeanBackward0>) tensor(0.1702, grad_fn=<AddBackward0>)
tensor(0.0605, grad_fn=<MeanBackward0>) tensor(0.1642, grad_fn=<AddBackward0>)
tensor(0.0582, grad_fn=<MeanBackward0>) tensor(0.154

In [7]:
torch.save({'model': net.state_dict()}, 'overfitted')
#points = grid_to_points(points[0:1])

In [9]:
from data.visualize import visualize_points
visualize_points(points[0].view(3, -1).t().contiguous())

In [12]:
net = net.eval()
pred = net(points[0].unsqueeze(0))[0].view(3, -1).t().contiguous().detach()
visualize_points(pred)